# Description

TODO.

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from IPython.display import display

import numpy as np
import pandas as pd
from sklearn.preprocessing import scale

import conf

# Settings

In [3]:
INPUT_FILEPATH = Path(
    conf.PHENOMEXCAN["SMULTIXCAN_EFO_PARTIAL_MASHR_ZSCORES_FILE"]
).resolve()
display(INPUT_FILEPATH)

input_filepath_stem = INPUT_FILEPATH.stem
display(input_filepath_stem)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/data/phenomexcan/gene_assoc/smultixcan-efo_partial-mashr-zscores.pkl')

'smultixcan-efo_partial-mashr-zscores'

In [4]:
# output dir for this notebook
RESULTS_DIR = Path(
    conf.RESULTS["DATA_TRANSFORMATIONS_DIR"],
    'z_score_std'
).resolve()
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

display(RESULTS_DIR)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/data_transformations/z_score_std')

# Load input file

In [5]:
data = pd.read_pickle(INPUT_FILEPATH).T

In [6]:
data.shape

(3749, 22515)

In [7]:
data.head()

gene_name,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,ENSG00000001460,ENSG00000001461,...,ENSG00000284240,ENSG00000284308,ENSG00000284395,ENSG00000284413,ENSG00000284418,ENSG00000284430,ENSG00000284452,ENSG00000284513,ENSG00000284526,ENSG00000284552
100001_raw-Food_weight,1.145442,0.618066,0.515724,0.280781,0.548127,0.045587,0.329995,0.109494,1.356741,1.474255,...,1.490410,0.230329,0.596503,0.519733,2.285074,0.124980,1.587903,1.522281,0.150938,1.010143
100002_raw-Energy,0.724557,1.028131,0.403596,0.253910,0.389877,0.197393,0.669649,0.041010,0.832120,0.954183,...,1.165679,0.111142,0.084263,1.229913,0.390190,0.505316,0.975901,1.817369,0.756393,0.729526
100003_raw-Protein,0.090876,2.218420,1.251359,0.879148,0.723469,0.777974,0.207873,0.536609,0.453969,1.286942,...,0.541370,0.734872,0.634674,1.316750,0.761859,1.276888,0.160988,0.346794,0.609476,0.222126
100004_raw-Fat,0.298165,0.762584,0.433091,0.352705,1.167250,0.578435,0.738983,0.565245,0.397189,0.192279,...,0.867217,0.540941,0.284347,1.661131,0.404078,1.248959,0.799771,1.443097,0.814969,0.545356
100005_raw-Carbohydrate,1.134347,0.934418,0.413466,0.051846,0.315952,0.046237,1.113674,0.319842,0.965217,0.919779,...,1.747265,0.496178,0.144053,0.701817,0.827677,0.587188,1.089338,2.001502,1.362716,1.490030


# Data preprocessing for clustering

In [8]:
data_stats = data.iloc[:, :10].describe()
display(data_stats)

gene_name,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,ENSG00000001460,ENSG00000001461
count,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000
mean,0.816770,0.876959,0.815741,0.878026,0.859195,0.859254,0.867773,0.815029,0.871436,0.890671
std,0.617591,0.665702,0.627319,0.683709,0.650563,0.645887,0.645205,0.625356,0.663147,0.672519
min,0.000061,0.000027,0.000129,0.000231,0.000002,0.000466,0.000025,0.000071,0.001368,0.000842
25%,0.332616,0.346044,0.325120,0.339387,0.350827,0.354458,0.366949,0.329258,0.360085,0.366680
50%,0.675571,0.743516,0.695028,0.725846,0.729148,0.723191,0.737108,0.695194,0.724852,0.739722
75%,1.162894,1.252868,1.156597,1.265937,1.228385,1.212088,1.241955,1.167031,1.233177,1.264729
max,4.850117,4.729789,5.508381,4.854216,6.282030,4.292140,5.074317,5.591921,4.185319,4.320655


## Standardize

In [9]:
scaled_data = pd.DataFrame(
    data=scale(data),
    index=data.index.copy(),
    columns=data.columns.copy()
)

In [10]:
scaled_data.shape

(3749, 22515)

In [11]:
scaled_data.head()

gene_name,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,ENSG00000001460,ENSG00000001461,...,ENSG00000284240,ENSG00000284308,ENSG00000284395,ENSG00000284413,ENSG00000284418,ENSG00000284430,ENSG00000284452,ENSG00000284513,ENSG00000284526,ENSG00000284552
100001_raw-Food_weight,0.532256,-0.388954,-0.478316,-0.873652,-0.478214,-1.259935,-0.833611,-1.128364,0.731919,0.867873,...,1.005188,-0.935278,-0.436995,-0.509571,2.475426,-1.066384,1.287210,1.193925,-1.041581,0.279640
100002_raw-Energy,-0.149331,0.227117,-0.657082,-0.912959,-0.721497,-1.024869,-0.307113,-1.237890,-0.059294,0.094451,...,0.502082,-1.115211,-1.200357,0.587988,-0.686398,-0.419785,0.269785,1.686974,-0.031378,-0.167471
100003_raw-Protein,-1.175520,2.015377,0.694506,0.001641,-0.208655,-0.125860,-1.022913,-0.445279,-0.629608,0.589313,...,-0.465159,-0.173582,-0.380112,0.722192,-0.066227,0.891941,-1.084969,-0.770138,-0.276510,-0.975917
100004_raw-Fat,-0.839835,-0.171834,-0.610058,-0.768442,0.473584,-0.434838,-0.199638,-0.399480,-0.715241,-1.038609,...,0.039676,-0.466355,-0.902183,1.254422,-0.663224,0.844460,-0.023022,1.061620,0.066355,-0.460911
100005_raw-Carbohydrate,0.514288,0.086326,-0.641346,-1.208541,-0.835145,-1.258929,0.381171,-0.791955,0.141437,0.043288,...,1.403133,-0.533932,-1.111255,-0.228166,0.043597,-0.280597,0.458369,1.994631,0.980272,1.044251


In [12]:
scaled_data_stats = scaled_data.iloc[:,:10].describe()
display(scaled_data_stats)

gene_name,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,ENSG00000001460,ENSG00000001461
count,3.749000e+03,3.749000e+03,3.749000e+03,3.749000e+03,3.749000e+03,3.749000e+03,3.749000e+03,3.749000e+03,3.749000e+03,3.749000e+03
mean,1.576878e-15,-4.472875e-16,-6.443973e-17,-2.225066e-15,5.155178e-16,3.297798e-16,-5.382612e-15,2.780385e-15,-2.026061e-15,-1.194030e-15
std,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00,1.000133e+00
min,-1.322587e+00,-1.317479e+00,-1.300330e+00,-1.284043e+00,-1.320866e+00,-1.329802e+00,-1.345099e+00,-1.303365e+00,-1.312203e+00,-1.323305e+00
25%,-7.840448e-01,-7.976321e-01,-7.821967e-01,-7.879241e-01,-7.815311e-01,-7.816585e-01,-7.763282e-01,-7.768957e-01,-7.711997e-01,-7.792504e-01
50%,-2.286597e-01,-2.004809e-01,-1.924531e-01,-2.226103e-01,-1.999248e-01,-2.106894e-01,-2.025445e-01,-1.916524e-01,-2.210722e-01,-2.244832e-01
75%,5.605168e-01,5.647562e-01,5.434256e-01,5.674386e-01,5.675686e-01,5.463496e-01,5.800187e-01,5.629563e-01,5.455647e-01,5.562791e-01
max,6.531648e+00,5.788393e+00,7.481468e+00,5.816391e+00,8.336710e+00,5.315700e+00,6.520570e+00,7.639693e+00,4.997877e+00,5.100882e+00


## Testing

In [13]:
assert np.all([np.isclose(scaled_data_stats.loc['mean', c], 0.0) for c in scaled_data_stats.columns])

In [14]:
assert np.all([np.isclose(scaled_data_stats.loc['std', c], 1.0, atol=1e-03) for c in scaled_data_stats.columns])

# Save

In [15]:
output_file = Path(
    RESULTS_DIR,
    f'z_score_std-{input_filepath_stem}.pkl',
).resolve()

display(output_file)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/data_transformations/z_score_std/z_score_std-smultixcan-efo_partial-mashr-zscores.pkl')

In [16]:
scaled_data.to_pickle(output_file)